# 다층퍼셉트론_MNIST_손글씨_예측모델
이번 실습을 통해서, MNIST 손글씨 예측을 하는 다층퍼셉트론을 텐서플로우로 구현해보도록 하겠습니다.

In [1]:
import tensorflow as tf
import numpy as np
from IPython.display import Image

# 다층퍼셉트론 구조
텐서플로우로 아래의 다층퍼셉트론을 구현해보도록 하겠습니다.

In [2]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

# MNIST 데이터 불러오기

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


학습 데이터는 총 **60000** 개의 샘플이 있습니다.  
테스트 데이터는 총 **10000** 개의 샘플이 있습니다.     
모든 데이터는 **28 * 28** 의 픽셀을 가지고 있습니다.  

MNIST 데이터는 아래의 그림처럼, 28*28의 픽셀 데이터입니다.  
각각의 픽셀은 흑백 사진과 같이 0부터 255까지의 그레이 스케일을 가지고 있습니다.

![0 from MNIST](https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/mnist_sample.png)

# 학습데이터에서 검증 데이터 분리하기
학습 중간마다 검증 데이터로 모델의 성능을 측정하면 다음과 같은 장점이 있습니다.  
1) 모델 학습이 제대로 진행되는 지 검증 정확도로 확인할 수 있습니다.  
2) 학습 정확도는 올라가는 데 검증 정확도가 안올라가거나 떨어질 시, 조기 종료를 구현할 수 있습니다.

In [5]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [6]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

train data has 50000 samples
every train data is 28 * 28 image


In [7]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

validation data has 10000 samples
every train data is 28 * 28 image


**0** 부터 **255** 까지의 그레이 스케일을 확인할 수 있습니다.

In [8]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


**0** 부터 **9**까지의 이미지에 해당하는 숫자를 확인할 수 있습니다.

In [9]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


테스트 데이터는 **10000** 개의 샘플을 가지고 있습니다.  
모든 테스트 데이터는 **28 * 28** 의 이미지입니다.  

In [10]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


# 데이터 구조 변경하기
다층퍼셉트론의 입력 레이어에 데이터를 넣기 위해서 2d tensor (28, 28)인 데이터를,  
1d tensor (28*28, 1)의  형태로 바꿔줍니다.  
이 말은 행렬 형태의 데이터를 배열 형태의 데이터로 변경한다는 의미와 같습니다.

In [11]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png", width=500, height=250)

In [12]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)


In [13]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

# 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고, 더 나은 성능을 구하도록 도와줍니다.  
MNIST 데이터의 모든 값은 0부터 255의 범위 안에 있으므로, 255로 값을 나눠줌으로써, 모든 값을 0부터 1 사이의 값으로 정규화합니다.  

In [14]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

# 실제값을 one hot encoding으로 변경하기
손실 함수에서 크로스 엔트로피를 계산하기 위해, 실제값은 one hot encoding 값으로 변겨합니다.

In [15]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [16]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# 텐서플로우 다층퍼셉트론 그래프 구현하기
텐서플로우로 아래의 다층퍼셉트론을 구현해보도록 하겠습니다.

In [17]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

입력 레이어와 출력레이어를 정의해줍니다.

In [18]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

다층퍼셉트론을 아래와 같이 구현해줍니다.

In [19]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits= tf.matmul(h2, w3) + b3
    
    return logits

다층퍼셉트론의 출력값을 logit으로 정의합니다.

In [20]:
logits = mlp(x)

손실함수는 logit과 실제값의 크로스 엔트로피를 줄이는 것을 목표로 설정합니다.

In [21]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y))

Adam Optimizer를 옵티마이저로 사용하도록 하겠습니다.

In [22]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

# 학습 및 테스트

In [23]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 30
batch_size = 1000
iteration = len(x_train) // batch_size

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        print("epoch: "+str(epoch)+", validation accuracy: " 
              + str(cur_val_acc) +', loss: '+str(avg_loss))
    
    # Test model
    preds = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))

epoch: 0, validation accuracy: 0.2683, loss: 9237.788642578123
epoch: 1, validation accuracy: 0.7878, loss: 318.60726390838624
epoch: 2, validation accuracy: 0.8595, loss: 16.967085990905762
epoch: 3, validation accuracy: 0.8762, loss: 10.046546058654783
epoch: 4, validation accuracy: 0.8801, loss: 7.77131690979004
epoch: 5, validation accuracy: 0.8914, loss: 6.301006126403808
epoch: 6, validation accuracy: 0.8756, loss: 5.5055615234375
epoch: 7, validation accuracy: 0.899, loss: 6.582991867065431
epoch: 8, validation accuracy: 0.9043, loss: 4.647599802017211
epoch: 9, validation accuracy: 0.9093, loss: 4.672785301208495
epoch: 10, validation accuracy: 0.9021, loss: 3.8992454147338877
epoch: 11, validation accuracy: 0.8668, loss: 8.740710778236389
epoch: 12, validation accuracy: 0.8736, loss: 7.349660973548889
epoch: 13, validation accuracy: 0.8625, loss: 6.908348550796508
epoch: 14, validation accuracy: 0.8631, loss: 35.530231952667236
epoch: 15, validation accuracy: 0.9079, loss: 9.4